In [2]:
import dvc.api
import pandas as pd

In [85]:
from torch import nn
import torch

In [3]:
import sys 
sys.path.append('../../')

In [4]:
import yaml

In [5]:
from main_multitask_multimodal import LLM_MultitaskMultimodal

In [6]:
config = yaml.safe_load(open('../../models/product_title_multitask_multimodal/version_1/config.yaml', 'r'))

In [7]:
config['model']['multitask_specs_dict']

{'clm_multimodal_clip2wishtitle': {'multimodal_embedding': {'input': [{'key': 'img_embedding',
     'proj_head': 'proj_head'}]}},
 'dlm_multimodal_wishtitlewclip2wishtitle': {'multimodal_embedding': {'input': [{'key': 'img_embedding',
     'proj_head': 'proj_head'}]}},
 'seqclf_multimodal_wishtitlewclip2pseudov121tax': {'multimodal_embedding': {'input': [{'key': 'img_embedding',
     'proj_head': 'proj_head'}]},
  'specs': {'clf_head': 'clf_head',
   'clf_weight_type': None,
   'label_map_file': 'datasets/taxonomy/wish_v1.2.1_newtax_allpaths.txt',
   'label_type': 'taxonomy'}},
 'emb_singlemodal_wishquery2googletitle': None,
 'clm_singlemodal_alititle2v121tax': None,
 'clm_singlemodal_wishtitle2pseudov121tax': None,
 'dlm_singlemodal_wishtitle': None,
 'emb_singlemodal_wishtitle2pseudov121tax': None,
 'emb_singlemodal_alititle2v121tax': None,
 'seqclf_singlemodal_alititle2v121tax': {'specs': {'clf_head': 'clf_head',
   'clf_weight_type': None,
   'label_map_file': 'datasets/taxonomy/wi

In [8]:
md = {'clm_multimodal_clip2wishtitle': {'multimodal_embedding': {'input': [{'key': 'img_embedding',
     'proj_head': 'proj_head'}]}},
 'dlm_multimodal_wishtitlewclip2wishtitle': {'multimodal_embedding': {'input': [{'key': 'img_embedding',
     'proj_head': 'proj_head'}]}},
 'seqclf_multimodal_wishtitlewclip2pseudov121tax': {'multimodal_embedding': {'input': [{'key': 'img_embedding',
     'proj_head': 'proj_head'}]},
  'specs': {'clf_head': 'clf_head',
   'clf_weight_type': None,
   'label_map_file': '/workspaces/multitask-llm-rnd/modelling/datasets/taxonomy/wish_v1.2.1_newtax_allpaths.txt',
   'label_type': 'taxonomy'}},
 'emb_singlemodal_wishquery2googletitle': None,
 'clm_singlemodal_alititle2v121tax': None,
 'clm_singlemodal_wishtitle2pseudov121tax': None,
 'dlm_singlemodal_wishtitle': None,
 'emb_singlemodal_wishtitle2pseudov121tax': None,
 'emb_singlemodal_alititle2v121tax': None,
 'seqclf_singlemodal_alititle2v121tax': {'specs': {'clf_head': 'clf_head',
   'clf_weight_type': None,
   'label_map_file': '/workspaces/multitask-llm-rnd/modelling/datasets/taxonomy/wish_v1.2.1_newtax_allpaths.txt',
   'label_type': 'taxonomy'}},
 'seqclf_singlemodal_wishtitle2pseudov121tax': {'specs': {'clf_head': 'clf_head',
   'clf_weight_type': None,
   'label_map_file': '/workspaces/multitask-llm-rnd/modelling/datasets/taxonomy/wish_v1.2.1_newtax_allpaths.txt',
   'label_type': 'taxonomy'}}}

In [9]:
model = LLM_MultitaskMultimodal.load_from_checkpoint(
    '../../models/product_title_multitask_multimodal/version_1/epoch=0-step=75000.ckpt', 
    multitask_specs_dict = md
)

INFO:root:Unused kwargs when getting t5-base: {'distance_func': 'cosine', 'loss_type': 'cross-entropy', 'margin': None, 'hidden_states_type': 'encoder-last', 'add_simcse': False, 'manual_loss_type': 'manual_mse', 'auto_task_weight': False, 'multitask_specs_dict': {'clm_multimodal_clip2wishtitle': {'multimodal_embedding': {'input': [{'key': 'img_embedding', 'proj_head': 'proj_head'}]}}, 'dlm_multimodal_wishtitlewclip2wishtitle': {'multimodal_embedding': {'input': [{'key': 'img_embedding', 'proj_head': 'proj_head'}]}}, 'seqclf_multimodal_wishtitlewclip2pseudov121tax': {'multimodal_embedding': {'input': [{'key': 'img_embedding', 'proj_head': 'proj_head'}]}, 'specs': {'clf_head': 'clf_head', 'clf_weight_type': None, 'label_map_file': '/workspaces/multitask-llm-rnd/modelling/datasets/taxonomy/wish_v1.2.1_newtax_allpaths.txt', 'label_type': 'taxonomy'}}, 'emb_singlemodal_wishquery2googletitle': None, 'clm_singlemodal_alititle2v121tax': None, 'clm_singlemodal_wishtitle2pseudov121tax': None, '

In [10]:
model.eval()
tmp = 2

# useful

In [60]:
df_attr = pd.read_csv(dvc.api.get_url(
    'data/wish_attr_extract/Amazon_FlatCat_AttrValDef.csv',
    repo='git@github.com:ContextLogic/multitask-llm-rnd.git'
))
df_attr['amazon_l1_flattype'] = (df_attr.L1_source + " > " + df_attr.product_type).str.lower().str.strip()
len(set(df_attr.L1_source)), len(set(df_attr.product_type)), len(set(df_attr.amazon_l1_flattype))

(37, 984, 995)

In [61]:
all_amazon_l1_flattypes = list(set(df_attr['amazon_l1_flattype']))

In [62]:
df_mapping = pd.read_excel(dvc.api.get_url(
    'data/wish_attr_extract/Wish_Amazon_Openai.xlsx',
    repo='git@github.com:ContextLogic/multitask-llm-rnd.git'
), sheet_name='All Completed')

In [63]:
df_mapping = df_mapping[~df_mapping.apth.isna()]

In [56]:
# set(df_mapping.wpath.apply(lambda x: x.split('/')[0].lower().strip()))

In [64]:
df_mapping['amazon_leaf'] = df_mapping.apth.apply(lambda x: x.replace("/", " > ").lower().strip())

In [65]:
all_amazon_matched_leaf = list(set(df_mapping['amazon_leaf']))

In [72]:
all_amazon_l1_flattypes_tensors = model.tokenizer(all_amazon_l1_flattypes, return_tensors='pt', padding=True)
all_amazon_matched_leaf_tensors = model.tokenizer(all_amazon_matched_leaf, return_tensors='pt', padding=True)

In [73]:
all_amazon_l1_flattypes_hidden_states = model.get_hidden_states(
    **all_amazon_l1_flattypes_tensors
)

In [74]:
all_amazon_matched_leaf_hidden_states = model.get_hidden_states(
    **all_amazon_matched_leaf_tensors
)

In [92]:
sims = nn.functional.normalize(all_amazon_matched_leaf_hidden_states, 1).mm(
    nn.functional.normalize(all_amazon_l1_flattypes_hidden_states, 1).T)

In [119]:
scores, indices = sims.topk(10)

In [120]:
results = {}

In [121]:
for ind, i in enumerate(indices.numpy()):
    results[all_amazon_matched_leaf[ind]] = [ 
        all_amazon_l1_flattypes[j] for j in i
    ]

In [122]:
df_res = pd.DataFrame(results).T.reset_index()

In [125]:
df_res.columns = ['amazon_leaf'] + [f'top_{i}_pred_amz_l1flat' for i in range(1, 11)]

In [126]:
df_res.to_excel('../../tmp_amazon_leaf_to_l1flat.xlsx')

In [127]:
df_res.sample(5).to_dict('records')

[{'amazon_leaf': 'sports & outdoors > exercise & fitness > clothing > men > sets > workout top & bottom sets',
  'top_1_pred_amz_l1flat': 'clothing > tracksuit',
  'top_2_pred_amz_l1flat': 'sports > sportwetsuit',
  'top_3_pred_amz_l1flat': 'clothing > suit',
  'top_4_pred_amz_l1flat': 'clothing > ethnicwear',
  'top_5_pred_amz_l1flat': 'clothing > sleepwear',
  'top_6_pred_amz_l1flat': 'clothing > jinbei',
  'top_7_pred_amz_l1flat': 'sports > sportracket',
  'top_8_pred_amz_l1flat': 'clothing > salwarsuitset',
  'top_9_pred_amz_l1flat': 'clothing > luggage',
  'top_10_pred_amz_l1flat': 'health > prescriptioneyewear'},
 {'amazon_leaf': 'sports & outdoors > clothing > men > shorts & pants > pants > snowboarding',
  'top_1_pred_amz_l1flat': 'clothing > shorts',
  'top_2_pred_amz_l1flat': 'clothing > pants',
  'top_3_pred_amz_l1flat': 'clothing > tights',
  'top_4_pred_amz_l1flat': 'clothing > underpants',
  'top_5_pred_amz_l1flat': 'clothing > overalls',
  'top_6_pred_amz_l1flat': 'cloth